# Image matching with Siamese networks
## Dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torch.utils.data import DataLoader
import utilities as utils 
from siameseDataloader import readDataFolder, dataSplits, MyImagenetDataset, EmbeddingDataset

import torch.nn as nn

from classifierModels import SimpleNetwork,Resnet20, EfficientNet,MobileNet,FCnet

In [2]:
import os
config = utils.load_config()
datadir = os.path.join('data','tiny-imagenet-200-copy','train')#config['DATASET']['root']
numclasses = config['DATASET']['numclasses']
im_size = config['DATASET']['im_size']
emb_size = config['MODEL']['embsize']

seed = config['seed']
np.random.seed(seed)

## Dataset for embs

In [3]:
if config['MODEL']['model']=='embeddings':
    train_dataset = EmbeddingDataset('trainembs.pkl')
    val_dataset = EmbeddingDataset('valembs.pkl')
    test_dataset = EmbeddingDataset('valembs.pkl')

## Dataset

In [4]:
dataset = readDataFolder(datadir,numclasses)
print(dataset)


Dataset ImageFolder
    Number of datapoints: 25000
    Root location: data\tiny-imagenet-200-copy\train


In [5]:
train_split,val_split,test_split = dataSplits(dataset,0.8,0.2,0)

In [6]:
train_transforms = transforms.Compose([
    transforms.Resize(im_size+4),
    transforms.CenterCrop(im_size),
    transforms.ToTensor(),
    #transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    transforms.Normalize([0.507, 0.4865, 0.4409], [0.2673, 0.2564, 0.2761])
])
test_transforms = transforms.Compose([
    transforms.Resize(im_size),
    transforms.CenterCrop(im_size),
    transforms.ToTensor(),
    #transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    transforms.Normalize([0.507, 0.4865, 0.4409], [0.2673, 0.2564, 0.2761])
])

train_dataset = MyImagenetDataset(train_split,train_transforms)
val_dataset = MyImagenetDataset(val_split,test_transforms)
test_dataset = MyImagenetDataset(test_split,test_transforms)

datasets = {'train':train_dataset,'val':val_dataset,'test':test_dataset}

## Dataloader

In [7]:
#dataloaders
dataloaders = {
    'train': DataLoader(train_dataset,shuffle=True,
                        num_workers=config['TRAIN']['numworkers'],
                        batch_size=config['TRAIN']['batchsize']),
    'val': DataLoader(val_dataset,shuffle=False,
                        num_workers=config['TRAIN']['numworkers'],
                        batch_size=config['TRAIN']['batchsize']),
    'test': DataLoader(test_dataset,shuffle=False,
                        num_workers=config['TRAIN']['numworkers'],
                        batch_size=config['TRAIN']['batchsize'])
}

## Model

In [8]:
if config['MODEL']['model'] == 'simple':
    model = SimpleNetwork(numclasses=numclasses,pretrained=config['MODEL']['pretrained'])
elif config['MODEL']['model'] == 'resnet':
    model = Resnet20(numclasses=numclasses)
elif config['MODEL']['model'] == 'mobilenet':
    model = MobileNet(numclasses=numclasses)
elif config['MODEL']['model'] == 'efficient':
    model = EfficientNet(numclasses=numclasses)
elif config['MODEL']['model']=='embeddings':
    model = FCnet(emb_size*2,1)
else:
    model = SimpleNetwork()

In [9]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

70418


In [10]:
data,labels = train_dataset[100]
out = model(data.reshape(1,3,im_size,im_size))
print(out)

top_p , top_class = torch.topk(out,1,dim=1)
equals = top_class.view(size=labels.shape) == labels
print(labels,top_class,equals)


tensor([[-0.0919,  0.0166,  0.0144, -0.1575, -0.1179, -0.0589,  0.1660, -0.1011,
          0.0425,  0.0645, -0.1261, -0.0803,  0.0768,  0.0817, -0.0996,  0.0634,
          0.0797, -0.0752,  0.0932, -0.0533,  0.0128,  0.0467, -0.0336,  0.1242,
         -0.0347,  0.0767,  0.1226,  0.0209, -0.0007,  0.0398,  0.0585, -0.1370,
         -0.0675,  0.0171, -0.0316,  0.0596, -0.0804, -0.0832,  0.0625, -0.0265,
         -0.1092,  0.0483, -0.0465, -0.0536, -0.0897, -0.0738, -0.0011,  0.0234,
          0.0394,  0.0353]], grad_fn=<AddmmBackward0>)
tensor(17) tensor([[6]]) tensor(False)


In [11]:
from torchview import draw_graph

# device='meta' -> no memory is consumed for visualization
model_graph = draw_graph(model, input_size=(1,3,64,64), device='meta')
model_graph.visual_graph

RuntimeError: Failed to run torchgraph see error message

## Train

In [ ]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()

config['TRAIN']['lr'] =0.0005
optimizer = optim.Adam(model.parameters(),lr = config['TRAIN']['lr'],weight_decay=0)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
device = config['TRAIN']['device']
model = model.to(device)

In [ ]:
loss_history ={'train':[],'val':[]}
acc_history ={'train':[],'val':[]}

In [ ]:

for epoch in range(config['TRAIN']['numepochs']):
    for mode in ['train','val']:
        loss_epoch=0
        count=0
        correct=0
        
        for i,(img,label) in enumerate(dataloaders[mode]):
            img = img.to(device)
            label = label.to(device)
            
            if mode=='train':
                model.train()
                
                optimizer.zero_grad()
                out = model(img)
                loss = criterion(out,label)
                loss.backward()
                optimizer.step()
                
            else:
                model.eval()
                with torch.no_grad():
                    out = model(img)
                    loss = criterion(out,label)
            
            # track total loss
            loss_epoch = loss_epoch+loss
            count = count + len(label)
            
            #top_p , top_class = torch.topk(out,1,dim=1)
            #equals = label-top_class.flatten()==0
            top_class = (out>=0.5)*1.0
            equals = label.flatten()-top_class.flatten()==0
            
            correct += torch.sum(equals)
            
        scheduler.step()

        loss_history[mode].append(loss_epoch.item()/count)
        acc_history[mode].append(correct.item()*1.0/count)
    
    print('Epoch: {}\t train loss:{:.5},\t val loss:{:.5},\t train acc:{:.5},\t val acc:{:.5}'.format(epoch,loss_history['train'][-1],loss_history['val'][-1],acc_history['train'][-1],acc_history['val'][-1]))
    
            
            

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,5))
ax[0].plot(loss_history['train'],'b-')
ax[0].plot(loss_history['val'],'r-')
ax[0].set_title('Loss')
ax[1].plot(acc_history['train'],'b-')
ax[1].plot(acc_history['val'],'r-')
ax[1].set_title('Acc')

# SVM

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

X_train, y_train = torch.load('saved\\trainembs_triplet.pkl')
X_val,y_val = torch.load('saved\\valembs_triplet.pkl')

X_train = np.array(X_train)
y_train =np.array(y_train).ravel()
X_val = np.array(X_val)
y_val = np.array(y_val).ravel()


In [ ]:
from sklearn import svm
clf = svm.SVC(C=0.5,probability=True)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=0.0)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
prediction_train = clf.predict(X_train)
prediction_val = clf.predict(X_val)

In [ ]:
acc_train = np.sum(prediction_train == y_train)/len(y_train)
acc_val = np.sum(prediction_val == y_val)/len(y_val)
print(acc_train,acc_val)

In [ ]:
out = clf.predict_proba(X_val)[:,1]

In [ ]:
threshs= np.linspace(0,1,11)
accs = []
sensitivity = []
specificity = []
f1 = []
precision = []
label = y_val
for thresh in threshs:
    conf = np.zeros((2,2))
    conf[1,1] = np.matmul(label.T,(out>=thresh)*1)
    conf[1,0] = np.matmul(label.T,(out<thresh)*1)
    conf[0,1] = np.matmul((1-label).T,(out>=thresh)*1)
    conf[0,0] = np.matmul((1-label).T,(out<thresh)*1)
    pos = np.sum(label)
    neg = np.sum(1-label)

    accs.append(((conf[1,1]+conf[0,0])/(pos+neg)))
    sensitivity.append((conf[0,0]/neg)) 
    specificity.append((conf[1,1]/pos))
    f1.append(((2*conf[0,0])/(2*conf[0,0]+conf[0,1]+conf[1,0])))
    precision.append((conf[0,0]/(conf[0,0]+conf[1,0])).item())
    

In [ ]:
accs = np.array(accs)
sensitivity = np.array(sensitivity)
specificity = np.array(specificity)
precision = np.array(precision)
f1 = np.array(f1)

m1 = np.argmax(accs)
m2 = np.argmax(f1)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,4))
ax[0].plot(threshs,accs)
ax[0].set_xlabel('threshold')
ax[0].set_ylabel('accuracy')
ax[1].plot(threshs,f1)
ax[1].set_xlabel('threshold')
ax[1].set_ylabel('f1-score')
ax[2].plot(1-specificity,sensitivity,'o',label='_nolabel_')
ax[2].set_xlabel('1-specificity')
ax[2].set_ylabel('sensitivity')
ax[2].plot(1-specificity[m1],sensitivity[m1],'ro',label='acc thresh:{}'.format(threshs[m1]))
ax[2].plot(1-specificity[m2],sensitivity[m2],'go',label='f1 thresh:{}'.format(threshs[m2]))
ax[2].legend()

In [ ]:
config['MODEL']['model'] = 'mobilenet_triple_emb'
name = "saved\\"+config['MODEL']['model']+"_"
torch.save(threshs,name+'threshs.pkl')
torch.save(accs,name+'accs.pkl')
torch.save(f1,name+'f1.pkl')
torch.save(specificity,name+'specificity.pkl')
torch.save(sensitivity,name+'sensitivity.pkl')
torch.save(precision,name+'precision.pkl')